In [20]:
import pandas as pd
import numpy as np

In [21]:
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [22]:
from IPython.display import HTML

In [23]:
from models import createModel
from animate import AnimateFeature

In [24]:
# import model structure from models.py file
model = createModel()

In [25]:
# load model weights
model_string = f"models/dice_checked_data_model/weights_epochs3"
model.load_weights(model_string)

In [26]:
x_train = np.load("./unnorm_no_shuffle_data/x_train.npy")
y_train = np.load("./unnorm_no_shuffle_data/y_train.npy")
x_val = np.load("./unnorm_no_shuffle_data/x_val.npy")
y_val= np.load("./unnorm_no_shuffle_data/y_val.npy")

# x_val = np.load("./cleaned_data/x_val.npy")
# y_val = np.load("./cleaned_data/y_val.npy")
# x_test = np.load("./cleaned_data/x_test.npy")
# y_test = np.load("./cleaned_data/y_test.npy")
# train_mu = np.load("./cleaned_data/train_mu.npy")
# train_std = np.load("./cleaned_data/train_std.npy")


In [27]:
y_val.shape

(42187, 3)

In [28]:
preds = model.predict(x_val[:,:,4:])

1319/1319 [==============================] - 2s 1ms/step


In [30]:
# undo normalization for the input, so we can pass it into play visualizer
# unadj_x_val = x_val
# unadj_x_val[:,:,5:] = (x_val[:,:,5:]*train_std[5:])+train_mu[5:]

unadj_x_val = x_val

In [31]:
unadj_x_val[:,:,6] += (53.3/2)

In [32]:
preds_repeat = np.tile(preds, (1,23)).reshape(-1,23,3)
val_with_preds = np.concatenate((unadj_x_val, preds_repeat), axis=2)


In [33]:
col_names = ['gameId', 'playId', 'frameId', 'nflId', 'team_indicator', 'adj_x', 'adj_y', 's', 'a', 'adj_o', 'adj_dir', 'no_sack_prob', 'sack_prob', 'pred_time']

val_with_preds_df = pd.DataFrame(val_with_preds.reshape(-1, len(col_names)), columns=col_names)
val_with_preds_df.gameId = val_with_preds_df.gameId.astype('int64')


In [34]:
val_with_preds_df['true_sack'] = np.tile(y_val[:,1], 23)
#np.repeat(y_val[:,1], 23)

In [35]:
val_with_preds_df.head()

,gameId,playId,frameId,nflId,team_indicator,adj_x,adj_y,s,a,adj_o,adj_dir,no_sack_prob,sack_prob,pred_time,true_sack
0,2021101704,566.0,6.0,38544.0,1.0,79.63,21.84,0.03,0.08,191.54,166.65,0.0,1.0,223.76358,0.0
1,2021101704,566.0,6.0,38553.0,2.0,81.62,26.89,0.22,1.65,354.94,120.85,0.0,1.0,223.76358,0.0
2,2021101704,566.0,6.0,40171.0,1.0,79.68,26.69,0.09,0.84,168.56,139.89,0.0,1.0,223.76358,0.0
3,2021101704,566.0,6.0,41619.0,2.0,80.97,23.81,0.16,1.05,336.78,151.74,0.0,1.0,223.76358,0.0
4,2021101704,566.0,6.0,42444.0,1.0,79.64,29.09,0.01,0.16,160.43,240.35,0.0,1.0,223.76358,0.0


In [36]:
true_sack_df = val_with_preds_df.query("true_sack == 1")
true_sack_df.gameId = true_sack_df.gameId.astype('int64')

/var/folders/59/2qy07m9n56j1hjrbkcj93rhc0000gn/T/ipykernel_4499/1917819778.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  true_sack_df.gameId = true_sack_df.gameId.astype('int64')


In [37]:
true_sack_df.drop_duplicates(['gameId', 'playId'])

,gameId,playId,frameId,nflId,team_indicator,adj_x,adj_y,s,a,adj_o,adj_dir,no_sack_prob,sack_prob,pred_time,true_sack
151,2021101704,566.0,12.0,47835.0,2.0,82.47,29.41,1.39,1.05,5.23,91.29,9.527681e-01,0.047232,127.371559,1.0
1400,2021101704,617.0,14.0,53953.0,1.0,73.61,10.38,0.57,2.49,188.62,80.82,0.000000e+00,1.000000,80.007263,1.0
2289,2021101704,680.0,16.0,47794.0,2.0,95.19,27.20,1.85,2.18,342.71,131.26,0.000000e+00,1.000000,89.775879,1.0
3202,2021101704,704.0,27.0,42901.0,1.0,63.95,29.10,1.83,0.43,193.74,5.14,0.000000e+00,1.000000,86.492188,1.0
4265,2021101704,746.0,40.0,44915.0,2.0,54.66,23.52,4.25,1.21,172.99,112.73,2.947295e-28,1.000000,-12.705788,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
965426,2021102407,2890.0,15.0,38592.0,1.0,51.99,35.88,1.52,4.03,243.89,12.38,6.476741e-30,1.000000,60.481167,1.0
966046,2021102407,2935.0,6.0,37266.0,2.0,33.70,29.26,0.81,2.22,322.45,178.18,0.000000e+00,1.000000,243.659363,1.0
967772,2021102407,3074.0,35.0,38843.0,2.0,85.65,21.24,1.24,2.52,188.62,272.13,0.000000e+00,1.000000,148.497833,1.0
967794,2021102407,3161.0,6.0,38542.0,2.0,34.59,26.97,0.31,1.27,205.64,168.00,0.000000e+00,1.000000,140.998093,1.0


In [38]:
# test_game_id = 2021101704
# test_play_id = 746.0
# good example of probability increasing as defender comes near

test_game_id = 2021102407
test_play_id = 2614.0

In [39]:

play_df = val_with_preds_df.query("gameId == @test_game_id and playId == @test_play_id")
print(f"play_df shape = {play_df.shape}")

play_df.head()

play_df shape = (782, 15)


,gameId,playId,frameId,nflId,team_indicator,adj_x,adj_y,s,a,adj_o,adj_dir,no_sack_prob,sack_prob,pred_time,true_sack
963194,2021102407,2614.0,6.0,37266.0,2.0,38.81,27.00,0.11,0.95,319.50,194.42,0.0,1.0,116.316727,0.0
963195,2021102407,2614.0,6.0,38592.0,1.0,32.16,8.02,0.18,0.42,70.35,339.14,0.0,1.0,116.316727,0.0
963196,2021102407,2614.0,6.0,39950.0,2.0,39.85,29.65,0.93,2.10,16.01,151.07,0.0,1.0,116.316727,0.0
963197,2021102407,2614.0,6.0,42391.0,1.0,32.05,22.27,0.15,0.58,160.29,315.43,0.0,1.0,116.316727,0.0
963198,2021102407,2614.0,6.0,43358.0,1.0,37.52,21.59,0.10,0.49,177.97,195.82,0.0,1.0,116.316727,0.0


In [40]:
animated_play = AnimateFeature(play_df)
HTML(animated_play.ani.to_jshtml())

In [18]:
animated_play = AnimateFeature(play_df)
HTML(animated_play.ani.to_jshtml())

In [48]:
# from importlib import reload

# def importOrReload(module_name, *names):
#     import sys

#     if module_name in sys.modules:
#         reload(sys.modules[module_name])
#     else:
#         __import__(module_name, fromlist=names)

#     for name in names:
#         globals()[name] = getattr(sys.modules[module_name], name)
        
# importOrReload("temp", "AnimatePlay")
